In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates


In [2]:
df = pd.read_csv("mpd_sp500.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# drop columns that has "maturity_target" , "lg_change_decr", and "lg_change_incr" in the column name; those are irrelevant for feature selection
df = df[df.columns.drop(list(df.filter(regex='maturity_target')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_decr')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_incr')))]
df = df[df.columns.drop(list(df.filter(regex='SP_adj_close')))]

# drop SP_lg_ret(%)	
df = df.drop(['SP_lg_ret(%)'], axis=1)
# df = df.drop(['SP_lg_pr'], axis=1)
df = df.drop(['VIX'], axis=1)

# Generate lagged variables from f1_mu to SP_lg_pr
lags = 6
for lag in range(1, lags+1):
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_ret_vol')+1]:
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('VIX')+1]: 
    for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_pr')+1]: 
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_ret(%)')+1]:    
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

df_lagged = df.copy()
# drop NA rows
df_lagged = df_lagged.dropna()
df_lagged.shape



# Define the target variable
start_colunm = df_lagged.columns.get_loc('f1_mu_lag1')
# end_column = df_lagged.columns.get_loc('VIX_lag6')
end_column = df_lagged.columns.get_loc('SP_lg_pr_lag6')
# end_column = df_lagged.columns.get_loc('SP_lg_ret(%)_lag6')

column_index = list(range(start_colunm, end_column+1))

X = df_lagged.iloc[:, column_index]
# y = df_lagged['SP_lg_ret(%)'] 
y = df_lagged['SP_lg_pr'] 

split_index = int(len(X)*0.75)
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_25171/709878160.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_25171/709878160.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_25171/709878160.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

In [3]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((428, 762), (143, 762), (428,), (143,))

In [4]:
# pred_lg_pr = pd.read_csv("Predicted_SP_lg_pr.csv")

pred_lg_pr = pd.read_csv("Rolling_Predicted_SP_lg_pr.csv")



In [5]:
# reset index for pred_lg_pr using the same index as y_test
pred_lg_pr = pred_lg_pr.set_index(y_test.index)
Backtest = X_test.copy()
Backtest['True_Lg_Price'] = y_test
Backtest['Predicted_Lg_Price'] = pred_lg_pr

In [6]:
# keep only the last two columns of Backtest
Backtest = Backtest[['True_Lg_Price', 'Predicted_Lg_Price']]

Backtest['True_Lg_Return'] = Backtest['True_Lg_Price'] - Backtest['True_Lg_Price'].shift(1)
Backtest['Predicted_Lg_Return'] = Backtest['Predicted_Lg_Price'] - Backtest['Predicted_Lg_Price'].shift(1)

# # if log return is positive, then 1; otherwise, 0
# Backtest['True_Return_Sign'] = np.where(Backtest['True_Lg_Return'] > 0, 1, 0)
# Backtest['Predicted_Sign'] = np.where(Backtest['Predicted_Lg_Return'] > 0, 1, 0)


因为我们用的都是lag1到lag9的数据，说明我们用这这些数据来预测下周的log price的情况
所以我们在同步与mpd数据的周频率来设置为我们买卖SP的时间，我们在频率周的第一天开盘时间enter position， 在频率周最后一天的收盘前决定是否exit position或者进行持仓。

In [7]:
# read the SP500 price data
SP500 = pd.read_csv("^SPX.csv")
SP500['Date'] = pd.to_datetime(SP500['Date'])
SP500.set_index('Date', inplace=True)


# # if log return is positive, then 1; otherwise, 0
# Backtest['True_Return_Sign'] = np.where(Backtest['True_Lg_Return'] > 0, 1, 0)
# Backtest['Predicted_Sign'] = np.where(Backtest['Predicted_Lg_Return'] > 0, 1, 0)

# Calculate the percentage change for the trading signal
# Backtest['True_Lg_Return_Shifted'] = Backtest['True_Lg_Return'].shift(1)
Backtest['Perc_Change'] = ((Backtest['Predicted_Lg_Return'] - Backtest['True_Lg_Return'].shift(1)) / Backtest['Predicted_Lg_Return']) * 100

# Define trading signal based on the percentage change
# Using 1 for Long, 0 for Short, and -1 for Hold
Backtest['Predicted_Return_Sign'] = np.where(Backtest['Perc_Change'] > 5, 1, np.where(Backtest['Perc_Change'] < -5, 0, -1))


start_date = Backtest.index[0]
end_date = Backtest.index[-1]
print(start_date, end_date)

# select SP500 price data between start_date and end_date
SP500 = SP500.loc[start_date:end_date]

# merge SP500 price data to Backtest
Combined = Backtest.merge(SP500, how='right', left_index=True, right_index=True)

2021-05-19 00:00:00 2024-02-07 00:00:00


In [8]:
# drop columns of High, Low, Close, and Volume
Final = Combined.drop(['Perc_Change','High', 'Low', 'Open', 'Adj Close', 'Volume'], axis=1)



In [9]:
# # # Forward fill Predicted_Lg_Return and columns
# # Combined['Predicted_Lg_Return'] = Combined['Predicted_Lg_Return'].ffill()
# # Combined['Predicted_Return_Sign'] = Combined['Predicted_Sign'].ffill()

# # Backward fill Predicted_Lg_Return and columns
# # Combined['Predicted_Lg_Return'] = Combined['Predicted_Lg_Return'].bfill()
# # Combined['Predicted_Return_Sign'] = Combined['Predicted_Sign'].bfill()

# Final = Combined.drop(['True_Lg_Return', 'True_Return_Sign', 'True_Lg_Return_Shifted', 'Predicted_Sign', 'Adj Close'], axis=1)

# # reoroder the columns
# # Final = Final[['True_Lg_Price',	'Predicted_Lg_Price', 'Predicted_Lg_Return', 'Predicted_Return_Sign', 'Close']]

# Final.head(20)

In [10]:
# drop NA rows expect the first row
Final2 = Final.iloc[[0]].append(Final.iloc[1:].dropna())
Final2.head(20)

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_25171/1143550365.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Final2 = Final.iloc[[0]].append(Final.iloc[1:].dropna())


,True_Lg_Price,Predicted_Lg_Price,True_Lg_Return,Predicted_Lg_Return,Predicted_Return_Sign,Close
Date,,,,,,
2021-05-19,8.322559,8.303837,NaN,NaN,-1.0,4115.680176
2021-05-26,8.341885,8.314411,0.019325,0.010574,-1.0,4195.990234
2021-06-02,8.344771,8.334829,0.002887,0.020418,1.0,4208.120117
2021-06-09,8.347484,8.339413,0.002712,0.004584,1.0,4219.549805
2021-06-16,8.348467,8.340361,0.000983,0.000948,0.0,4223.700195
2021-06-23,8.352752,8.340716,0.004286,0.000355,0.0,4241.839844
2021-06-30,8.365789,8.344679,0.013036,0.003963,0.0,4297.500000
2021-07-07,8.379798,8.356788,0.014010,0.012109,0.0,4358.129883
2021-07-14,8.383502,8.370084,0.003703,0.013296,0.0,4374.299805


In [11]:
# Final2.to_csv("Signal_List2.csv")

### Backtest

In [12]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [13]:
# initialize parameters
initial_balance = 10000000  # initial balance
# margin_ratio = 0.1  # margin ratio
commission_rate = 0.001  # transaction fee
# contract_multiplier = 100  # contract multiplier of futures
# balance_usage_ratio = 0.1  # balance usage ratio

# balance_usage_ratio / margin_ratio = leverage_ratio

trade_times = 0
right_times = 0
accumulated_earnings = 0
accumulated_losses = 0

# initialize account information
current_balance = initial_balance
# frozen_balance = 0
open_positions = {'Direction':'No_Position'}  # current position records
order_records = []  # order records 
account_values = []  # account values record

In [14]:
# # Update the signal calculation function to reverse the signal indications
# def calculate_signal(data):
#     data_copy = data.copy()  
#     if data_copy['Predicted_Return_Sign'].iloc[-1] == 0.0:
#         return "Long"  # Now indicates a "Long" signal when the last value is 0.0
#     elif data_copy['Predicted_Return_Sign'].iloc[-1] == 1.0:
#         if 0.0 in data_copy['Predicted_Return_Sign'].values:
#             return "Short"  # Now indicates a "Short" signal when the last value is 1.0 and there's a later 0.0
#         else:
#             return "No_Signal_Update"  # If there are no later 0.0, keep it as "No_Signal_Update"
#     else:
#         return "No_Signal_Update"

In [15]:
# Update the signal calculation function to consider future signals
def calculate_signal(data):
    data_copy = data.copy()  
    if data_copy['Predicted_Return_Sign'].iloc[-1] == 1.0:
        return "Long"
    elif data_copy['Predicted_Return_Sign'].iloc[-1] == -1.0:
        if 1.0 in data_copy['Predicted_Return_Sign'].values:
            return "Short"  # If there is a later 1.0, consider the current -1.0 as "Short"
        else:
            return "No_Signal_Update"  # If there are no later 1.0, keep it as "No_Signal_Update"
    else:
        return "No_Signal_Update"

In [16]:
# # Update the signal calculation function to consider future signals
# def calculate_signal(data):
#     data_copy = data.copy()  
#     if data_copy['Predicted_Return_Sign'].iloc[-1] == 1.0:
#         return "Long"
#     elif data_copy['Predicted_Return_Sign'].iloc[-1] == 0.0:
#         if 1.0 in data_copy['Predicted_Return_Sign'].values:
#             return "Short"  # If there is a later 1.0, consider the current 0.0 as "Short"
#         else:
#             return "No_Signal_Update"  # If there are no later 1.0, keep it as "No_Signal_Update"
#     else:
#         return "No_Signal_Update"

In [17]:
def open_position(window_data, signal):
    global current_balance, open_positions, order_records, trade_times, right_times, accumulated_earnings, accumulated_losses

    latest_price = window_data['Close'].iloc[-2]
    position_size = math.floor(current_balance / latest_price)


    open_positions = {
        'Symbol': 'SP500',
        'PositionSize':position_size,
        'AveragePrice':latest_price, 
        'Direction':signal
    }

    order_records.append({
        'DateTime':window_data.index[-1], 
        'Action':'Open_Position', 
        'Symbol': 'SP500',
        'Price':latest_price, 
        'Direction':signal
    })

In [18]:
def close_position(window_data):
    global current_balance, open_positions, order_records, trade_times, right_times, accumulated_earnings, accumulated_losses

    latest_price = window_data['Close'].iloc[-1]

    if open_positions['Direction'] == "Long":
        profit_loss = (window_data['Close'].iloc[-1] - open_positions['AveragePrice']) * open_positions['PositionSize']
        profit_loss -= commission_rate * open_positions['PositionSize'] * window_data['Close'].iloc[-1]
    elif open_positions['Direction'] == "Short":
        profit_loss = (open_positions['AveragePrice'] - window_data['Close'].iloc[-1]) * open_positions['PositionSize']
        profit_loss -= commission_rate * open_positions['PositionSize'] * window_data['Close'].iloc[-1]
    else:
        profit_loss = 0

    current_balance += profit_loss

    open_positions = {
        'Symbol': 'SP500',
        'PositionSize': 0,
        'AveragePrice':  0,
        'Direction': 'No_Position'
    }

    order_records.append({
        'DateTime':window_data.index[-1], 
        'Action':'Close_Position', 
        'Symbol': 'SP500',
        'Price':latest_price, 
        'Quantity':open_positions['PositionSize'],
        'ProfitLoss':profit_loss,
        'Direction':open_positions['Direction']
    })


    if profit_loss > 0:
        right_times += 1
        accumulated_earnings += profit_loss
    else:
        accumulated_losses += profit_loss
    
    trade_times += 1

In [19]:
# for i in range(len(Final)):
#     window_data = Final.iloc[:i+1]

for i in range(len(Final2)):
    window_data = Final2.iloc[:i+1]

##
    if len(window_data) < 2:  # Ensure there are at least 2 rows to avoid IndexError
        continue  # Skip to the next iteration if not enough data
##

    # signal = calculate_signal(window_data)
    # if (signal == "Long" or signal == "Short") and open_positions['Direction'] == "No_Position":
    #     open_position(window_data, signal)
    # elif signal == "Long" and open_positions['Direction'] == "Short":
    #     close_position(window_data)
    #     open_position(window_data, signal)
    # elif signal == "Short" and open_positions['Direction'] == "Long":
    #     close_position(window_data)
    #     open_position(window_data, signal)
    # else:
    #     pass

for i in range(len(Final)):
    window_data = Final.iloc[:i+1]
    signal = calculate_signal(window_data)
    if signal == "Long" and open_positions['Direction'] == "No_Position":
        open_position(window_data, signal)
    elif signal == "Short" and open_positions['Direction'] == "Long":
        # Close the long position if a short signal is received, instead of opening a short position
        close_position(window_data)
    # Removed the condition to open a short position
    else:
        pass

    latest_price = window_data['Close'].iloc[-1]

    if open_positions['Direction'] == "Long":
        profit_loss = (latest_price - open_positions['AveragePrice']) * open_positions['PositionSize']
    elif open_positions['Direction'] == "Short":
        profit_loss = (open_positions['AveragePrice'] - latest_price) * open_positions['PositionSize']
    else:
        profit_loss = 0
    
    total_balance = current_balance + profit_loss

    account_values.append({
        'DateTime':window_data.index[-1],
        'TotalBalance':total_balance,
        'CurrentBalance':current_balance,
        'ProfitLoss':profit_loss,
        'LatestPrice':latest_price,
        'Direction':open_positions['Direction'],
        'NetBalance':round(total_balance/initial_balance,2),
        'NetPrice':round(latest_price/window_data['Close'].iloc[0],2),
        'StrategyReturn':round(total_balance/initial_balance-1,2),
        'SP500_Return':round(latest_price/window_data['Close'].iloc[0]-1,2)
    })


    if current_balance + profit_loss < 0:
        close_position(window_data)

In [20]:
df_account_values = pd.DataFrame(account_values)
df_account_values.set_index('DateTime', inplace=True)
df_account_values.to_csv("Account_Values4.csv")

In [21]:
df_account_values.head(20)

,TotalBalance,CurrentBalance,ProfitLoss,LatestPrice,Direction,NetBalance,NetPrice,StrategyReturn,SP500_Return
DateTime,,,,,,,,,
2021-05-19,1.000000e+07,10000000.0,0.000000,4115.680176,No_Position,1.00,1.00,0.00,0.00
2021-05-20,1.000000e+07,10000000.0,0.000000,4159.120117,No_Position,1.00,1.01,0.00,0.01
2021-05-21,1.000000e+07,10000000.0,0.000000,4155.859863,No_Position,1.00,1.01,0.00,0.01
2021-05-24,1.000000e+07,10000000.0,0.000000,4197.049805,No_Position,1.00,1.02,0.00,0.02
2021-05-25,1.000000e+07,10000000.0,0.000000,4188.129883,No_Position,1.00,1.02,0.00,0.02
2021-05-26,1.000000e+07,10000000.0,0.000000,4195.990234,No_Position,1.00,1.02,0.00,0.02
2021-05-27,1.000000e+07,10000000.0,0.000000,4200.879883,No_Position,1.00,1.02,0.00,0.02
2021-05-28,1.000000e+07,10000000.0,0.000000,4204.109863,No_Position,1.00,1.02,0.00,0.02
2021-06-01,1.000000e+07,10000000.0,0.000000,4202.040039,No_Position,1.00,1.02,0.00,0.02
